xxx yolos

what do we need for yolo?
get all the labeld images?

transfrome labels?

weights?


In [16]:
pip install datasets

  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached propcache-0.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.5 MB/s eta 0:00:008.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 11.9 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached propcache-0.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (232 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
Note: you may need to restart the kernel to use updated packages

In [13]:
from transformers import AutoModelForObjectDetection,AutoImageProcessor
from PIL import Image, ImageDraw

import pandas as pd
import torch
import cv2 as cv
import numpy as np
import shutil
import os
import json


In [ ]:
image_processor = AutoImageProcessor.from_pretrained("hustvl/yolos-base")
model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-base")

In [ ]:
img = Image.open("tomo_00e463/slice_0235.jpg")
test_img=image_processor(images=img, return_tensors="pt")


In [ ]:
img

In [ ]:

outputs = model(**test_img)
#model.bbox_predictor(**test_img)

In [ ]:
target_sizes = torch.tensor([img.size[::-1]])

results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[

    0

]

In [ ]:
from transformers import YolosFeatureExtractor, YolosForObjectDetection

grayscale_image = Image.open("tomo_00e463/slice_0235.jpg").convert("L")

# Convert to RGB
rgb_image = Image.merge("RGB", (grayscale_image, grayscale_image, grayscale_image))

# Save the RGB image
rgb_image.save("rgb_test_img.jpg")
img = Image.open("rgb_test_img.jpg")

feature_extractor = YolosFeatureExtractor.from_pretrained('hustvl/yolos-base')
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-base')

inputs = feature_extractor(images=img, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
logits = outputs.logits
bboxes = outputs.pred_boxes

In [ ]:
bboxes

In [ ]:
target_sizes = torch.tensor([[img.size[1], img.size[0]]])
results = image_processor.post_process_object_detection(outputs, threshold=0.005, target_sizes=target_sizes)[0]


In [ ]:
results

In [ ]:
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):

    box = [round(i, 2) for i in box.tolist()]

    print(

        f"Detected {model.config.id2label[label.item()]} with confidence "

        f"{round(score.item(), 3)} at location {box}"

    )

In [ ]:
draw = ImageDraw.Draw(img)

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):

    box = [round(i, 2) for i in box.tolist()]

    x, y, x2, y2 = tuple(box)

    draw.rectangle((x, y, x2, y2), outline="red", width=1)

    draw.text((x, y), model.config.id2label[label.item()], fill="white")

img

---
## Fine Tune/ Transfer Modell

In [3]:
df=pd.read_csv("train_labels.csv")

In [18]:
df.head(2)

,row_id,tomo_id,Motor axis 0,Motor axis 1,Motor axis 2,Array shape (axis 0),Array shape (axis 1),Array shape (axis 2),Voxel spacing,Number of motors
0,0,tomo_003acc,-1.0,-1.0,-1.0,500,1912,1847,6.5,0
1,1,tomo_00e047,169.0,546.0,603.0,300,959,928,15.6,1


In [40]:
df["Voxel spacing"].value_counts()

Voxel spacing
13.1    291
15.6    177
19.7    104
16.8     85
16.1     40
13.3     18
19.3     10
19.2      7
6.5       3
13.2      2
Name: count, dtype: int64

In [33]:
print(df[df["Motor axis 0"]!=-1]["Motor axis 0"].nsmallest(5))
print(df[df["Motor axis 1"]!=-1]["Motor axis 1"].nsmallest(5))
print(df[df["Motor axis 2"]!=-1]["Motor axis 2"].nsmallest(5))

366     0.0
291     3.0
48     16.0
656    21.0
72     24.0
Name: Motor axis 0, dtype: float64
474    59.0
694    64.0
422    67.0
584    72.0
421    80.0
Name: Motor axis 1, dtype: float64
689    30.0
380    41.0
26     45.0
25     54.0
97     58.0
Name: Motor axis 2, dtype: float64


In [12]:
print(df["Array shape (axis 0)"].unique())
print(df["Array shape (axis 1)"].unique())
print(df["Array shape (axis 2)"].unique())

[500 300 600 800 400 494]
[1912  959  924  960  928 1024  927  956]
[1847  928  956  960 1440  959  924]


In [9]:
#print(len(df[df["Array shape (axis 1)"]!=959]))
print(len(df[df["Array shape (axis 1)"]!=959]))
print(len(df[df["Array shape (axis 2)"]!=928]))

663
281


In [34]:
print(len(df[df["Motor axis 0"]!=-1]))
print(len(df[df["Motor axis 1"]!=-1]))
print(len(df[df["Motor axis 2"]!=-1]))

451
451
451


In [39]:
print(len(df[df["Motor axis 0"]!=-1]["tomo_id"].unique()))
print(len(df[df["Motor axis 1"]!=-1]["tomo_id"].unique()))
print(len(df[df["Motor axis 2"]!=-1]["tomo_id"].unique()))

362
362
362


In [28]:
df[df["Motor axis 0"]==466]

,row_id,tomo_id,Motor axis 0,Motor axis 1,Motor axis 2,Array shape (axis 0),Array shape (axis 1),Array shape (axis 2),Voxel spacing,Number of motors
692,692,tomo_ec607b,466.0,458.0,301.0,500,928,960,13.1,2


In [23]:
df[df["Motor axis 2"]!=-1]["Motor axis 2"].min()

np.float64(30.0)

In [4]:
df_bbox=df.iloc[:,1:6]
labels=df_bbox.values
labels

array([['tomo_003acc', -1.0, -1.0, -1.0, 500],
       ['tomo_00e047', 169.0, 546.0, 603.0, 300],
       ['tomo_00e463', 235.0, 403.0, 137.0, 500],
       ...,
       ['tomo_fea6e8', -1.0, -1.0, -1.0, 300],
       ['tomo_ff505c', 111.0, 816.0, 678.0, 300],
       ['tomo_ff7c20', -1.0, -1.0, -1.0, 800]],
      shape=(737, 5), dtype=object)

In [37]:
# Create YOLO format label
# YOLO format: <class> <x_center> <y_center> <width> <height>
# Values are normalized to [0, 1]
# {"file_name": "0001.png", "objects": {"bbox": [[302.0, 109.0, 73.0, 52.0]], "categories": [0]}}
BBOXS_SIZE = 70
detections = []
detections_empty = []
trust = 4
rng = np.random.default_rng(42)
meta_train = []
meta_test = []
size_empty = 3

for a in labels:
    if a[1] != -1:
        for b in np.arange(int(a[1])-trust, int(a[1])+trust+1):
            if b < 0:
                continue
            width = BBOXS_SIZE
            height = BBOXS_SIZE
            if BBOXS_SIZE/2 > a[-2]:
                width = (a[-2])*2
            if BBOXS_SIZE/2 > a[2]:
                height = (a[2]-1)*2
            detections.append({"file_name": f"{a[0]}/slice_{b:04d}.jpg", "object": {
                              "bbox": [0, a[-2], a[2], width, height], "categories": [0]}})
    else:
        random_slice = rng.integers(low=0, high=a[-1], size=size_empty)

        for c in random_slice:
            detections_empty.append(
                {"file_name": f"{a[0]}/slice_{c:04d}.jpg", "object": {"bbox": [], "categories": []}})

print(len(detections_empty+detections))

print(f"portion of empty = {len(detections_empty)/len(detections)}")

# collect train metadata.jason
for i in detections:
    if i["object"]["bbox"][1] >= 0:
        source = "train/"+i["file_name"]
        destination = "data/train/"+i["file_name"]

        if not os.path.isdir(os.path.dirname(destination)):
            os.mkdir(os.path.dirname(destination))

        shutil.copyfile(source, destination)
        meta_train.append(i)

for i in detections_empty:

    source = "train/"+i["file_name"]
    destination = "data/train/"+i["file_name"]

    if not os.path.isdir(os.path.dirname(destination)):
        os.mkdir(os.path.dirname(destination))

    shutil.copyfile(source, destination)
    meta_train.append(i)


# Train test split
split = 0.1
size_det = len(detections)
size_empt = len(detections_empty)

test_split_det = int(size_det*split)
test_split_empt = int(size_empt*split)

indices_test_det = rng.integers(low=0, high=size_det, size=test_split_det)
indices_test_empt = rng.integers(low=0, high=size_empt, size=test_split_empt)


# copy test files


for i in indices_test_det:
    if detections[i]["object"]["bbox"][1] >= 0:
        source = "train/"+detections[i]["file_name"]
        destination = "data/test/"+detections[i]["file_name"]

        if not os.path.isdir(os.path.dirname(destination)):
            os.mkdir(os.path.dirname(destination))

        shutil.copyfile(source, destination)
        meta_test.append(detections[i])


for i in indices_test_empt:

    source = "train/"+detections_empty[i]["file_name"]
    destination = "data/test/"+detections_empty[i]["file_name"]

    if not os.path.isdir(os.path.dirname(destination)):
        os.mkdir(os.path.dirname(destination))

    shutil.copyfile(source, destination)
    meta_test.append(detections_empty[i])


# write to jason for hugginface

with open("data/train/metadata.json", "w") as outfile:
    json.dump(meta_train, outfile)


with open("data/test/metadata.json", "w") as outfile:
    json.dump(meta_test, outfile)


# Create YAML configuration file for YOLO
"""yaml_content = {
    'path': yolo_dataset_dir,
    'train': 'images/train',
    'val': 'images/val',
    'names': {0: 'motor'}
}"""
print(len(indices_test_det)+len(indices_test_empt))


4912
portion of empty = 0.21164282190429207
490


In [46]:
from datasets import load_dataset
dataset=[]
# Load dataset
dataset = load_dataset("imagefolder", data_dir="data/")

#id2label = {0: "motor"}

In [47]:
if "validation" not in dataset:

    split = dataset["train"].train_test_split(0.075, seed=1337)

    dataset["train"] = split["train"]

    dataset["validation"] = split["test"]

In [52]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4386
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 462
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 356
    })
})

In [56]:
dataset["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=928x959>,
 'label': 433}

In [57]:
dataset["train"][1]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=928x960>,
 'label': 262}

In [ ]:

MAX_SIZE = IMAGE_SIZE

image_processor = AutoImageProcessor.from_pretrained(

    "hustvl/yolos-base",

    do_resize=True,

    size={"max_height": MAX_SIZE, "max_width": MAX_SIZE},

    do_pad=True,

    pad_size={"height": MAX_SIZE, "width": MAX_SIZE},

)
